<font size="5.5"><u><i>Convert Strain to Image</i></u></font>

<font size="4">Script to convert strain samples (time series) to image samples (TF Morlet wavelet scalograms).</font>
<br/>
<font size="4">Author: Manuel David Morales</font>

<font size="4">*This code has been adapted to be run on Google Colaboratory*</font>

In [ ]:
# Mount Google Drive repository

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## 1) Library imports

In [ ]:
# Data analysis
import numpy as np
import pandas as pd

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

# Scientific computing
import scipy.misc
from scipy import signal

# Files/folders management
import os, glob, io

# To read csv files
import csv

# Garbage collector
import gc

# Image management
from PIL import Image

## 2) Input parameters

In [ ]:
# Interferometer for noise data
# --------------------------------------------------------------------
detector = "H1"    # Options: "L1", "H1", "V1"
# --------------------------------------------------------------------

# Class of samples to convert
# --------------------------------------------------------------------
sample_class = "3"    # Options: "1", "2", "3"

# Work with CCSNe general relativistic waveforms?
# --------------------------------------------------------------------
GRwf_cond = 1       # Options: 0 (no), 1 (yes)
# --------------------------------------------------------------------

# For GRwf_cond = 1, list CCSNe waveform models
# --------------------------------------------------------------------
wf_models = ["Andresen_2019", "Morozova_2018", "Cerda-Duran_2013"]
# --------------------------------------------------------------------

# For GRwf_cond = 1, set the distance (in kiloparsecs)
# --------------------------------------------------------------------
distance = 1     # Options: 1, 5, 10 (for now)
# --------------------------------------------------------------------

## 3) Read files

In [ ]:
# ------> Specify folder location

if GRwf_cond = 0:
  data_dir = '/content/drive/MyDrive/Colab Notebooks/GitHub/CCSNeHFGW_ResNetClass/Datasets/PhenWf/'
elif GRwf_cond = 1:
  data_dir = '/content/drive/MyDrive/Colab Notebooks/GitHub/CCSNeHFGW_ResNetClass/Datasets/GenRelWf/Distance_' + str(distance) + 'Kpc/'

print("Strain datasets are located at:", data_dir)
print("")

Strain datasets are located at: /content/drive/MyDrive/Colab Notebooks/GitHub/CCSNeHFGW_ResNetClass/Datasets/NumWf/Distance_10Kpc/



In [ ]:
# ---------------------------------------------------------------
# ------> Strain sample and log data files reader
# ---------------------------------------------------------------

def load_straindata(folder_path, class_samples, class_logdata):

    """
    Function to load strain samples for a given GPS segment.

    INPUT:
            folder_path    -> Folder path
            class_samples  -> Dictionary, to save strain
                              samples for each class (key)
            class_logdata  -> Dictionary, to save log
                              data for each class (key)

    OUTPUT:
            class_samples  -> Updated samples dictionary
            class_logdata  -> Updated log dictionary
            Num_samples    -> No. of loaded samples
    """

    os.chdir(folder_path)
    #print(folder_path)

    # Initialize No. of samples count
    Num_samples = 0

    # Loop: Waveform class subfolders
    # ---------------------------------
    for subfolder in glob.glob("wfclass_*"):

        print("=======> SCANNING", subfolder, "SUBFOLDER")

        class_label = subfolder[-1]

        # *********************************************************************
        # Condition: Phen. or Gen. Rel. waveforms?
        if GRwf_cond == 0:
          os.chdir(folder_path + "/" + subfolder)
          print(folder_path + "/" + subfolder)

        elif GRwf_cond == 1:
          path = folder_path + "/" + subfolder + "/" + wf_models[int(class_label)-1]
          os.chdir(path)
          print(path)
        # *********************************************************************

        windows_onelabel = []

        logdata_onelabel = []

        # Loop: Strain window samples
        # -----------------------------
        for file in glob.glob("sample_strain_*"):

            #print("***** READING FILE", file, " *****")

            # Extract sample number from file
            # Remark: Some ordinal numbers could be not present, because
            # in strain dataset generation, samples of SNR values outside
            # of an acceptation range are removed (SNR>100)
            Num_TFsample = int(file[-10:-4])

            # -----> Load log data
            # ----------------------

            # Remark: Read and update log data file is important, because
            # reading of strain samples does not following the same order as
            # the number of strain samples files. Then, TF samples will
            # be not enumerated following the order of the strain samples.

            with open("log.dat") as csv_file:

                csv_reader = csv.reader(csv_file, delimiter=',')
                line_count = 0

                for row in csv_reader:

                    if line_count == 0:
                        #print(f'Data columns: {", ".join(row)}')
                        line_count += 1

                    elif line_count == Num_TFsample + 2:
                        t_inj = row[0]
                        jitter = row[1]
                        wf_SNR = row[2]

                        # *****************************************************
                        # Condition: Phen. or Gen. Rel. waveforms?
                        if GRwf_cond == 0:
                          Slope = row[3]
                          f_ini = row[4]
                          f_end = row[5]
                          wf_duration = row[6]

                        elif GRwf_cond == 1:
                          wf_duration = row[3]
                        # *****************************************************

                        break

                    line_count += 1

            # *****************************************************************
            # Condition: Phen. or Gen. Rel. waveforms?
            if GRwf_cond == 0:
              logdata_onelabel.append([str(Num_TFsample+1), t_inj, jitter, wf_SNR, Slope, f_ini, f_end, wf_duration])
            elif GRwf_cond == 1:
              logdata_onelabel.append([str(Num_TFsample+1), t_inj, jitter, wf_SNR, wf_duration])
            # *****************************************************************

            # -----> Load strain data
            # -------------------------

            with open(file) as csv_file:

                sample = pd.read_csv(file)
                sample_arr = sample.to_numpy()
                windows_onelabel.append(sample_arr)

                Num_samples += 1

        class_samples["class " + class_label] = windows_onelabel
        class_logdata["class " + class_label] = logdata_onelabel

    return class_samples, class_logdata, Num_samples

In [ ]:
# Initialize list for segments GPS initial times
GPS_seg = []

# Initialize list for strain data by GPS segments
strain_segments = []

# Initialize list for log data by GPS segments
logdata_segments = []

# Initialize No. of total injections count
Num_samples_total = 0

# ------> Scan folders

os.chdir(data_dir)

# ------> Loop: GPS segment folders
for folder in glob.glob(detector + "*"):

    if "_TF_samples" in folder:
      continue
    else:

      print("SCANNING", folder, "FOLDER")
      print("------------------------------------------")

      GPS_seg.append(folder[-10:])

      # Initialize dictionary for strain data

      class_samples = {}

      # Initialize dictionary for log adata

      class_logdata = {}

      # Call function load_straindata
      class_samples, class_logdata, Num_samples = load_straindata(data_dir + folder, class_samples, class_logdata)

      strain_segments.append(class_samples)
      logdata_segments.append(class_logdata)

      Num_samples_total += Num_samples

      print("")

print("***** Total number of loaded samples: ", Num_samples_total)

SCANNING H1_1256783872 FOLDER
------------------------------------------
=======> SCANNING wfclass_1 SUBFOLDER
/content/drive/MyDrive/Colab Notebooks/GitHub/CCSNeHFGW_ResNetClass/Datasets/NumWf/Distance_10Kpc/H1_1256783872/wfclass_1/Andresen_2019
=======> SCANNING wfclass_3 SUBFOLDER
/content/drive/MyDrive/Colab Notebooks/GitHub/CCSNeHFGW_ResNetClass/Datasets/NumWf/Distance_10Kpc/H1_1256783872/wfclass_3/Cerda-Duran_2013
=======> SCANNING wfclass_2 SUBFOLDER
/content/drive/MyDrive/Colab Notebooks/GitHub/CCSNeHFGW_ResNetClass/Datasets/NumWf/Distance_10Kpc/H1_1256783872/wfclass_2/Morozova_2018

***** Total number of loaded samples:  1533


In [ ]:
# ------> Some checks

print("Key of dictionaries in strain_segments lists:")
print("-----------------------------------------------")
for i in range(len(strain_segments)):
  print(strain_segments[i].keys())

print("")

print("Key of dictionaries in logdata_segments lists:")
print("-----------------------------------------------")
for i in range(len(strain_segments)):
  print(logdata_segments[i].keys())

# Remark: In strain_segments[n] and logdat_segments[n],
#         n value is the folder's number of a strain GPS segment

Key of dictionaries in strain_segments lists:
-----------------------------------------------
dict_keys(['class 1', 'class 3', 'class 2'])

Key of dictionaries in logdata_segments lists:
-----------------------------------------------
dict_keys(['class 1', 'class 3', 'class 2'])


## 4) Data preparation

Hereinafter, the analysis is performed only for the selected class

In [ ]:
# ------> Initial checks

print("Count of samples in segments of class", sample_class, ":")
print("-----------------------------------------")
print("")

for i in range(len(strain_segments)):
    print("---> GPS Segment", GPS_seg[i])
    #print("****************")
    print("Strain files: ", len(strain_segments[i]["class " + sample_class]))
    print("Log data files: ", len(logdata_segments[i]["class " + sample_class]))
    print("")


Count of samples in segments of class 3 :
-----------------------------------------

---> GPS Segment 1256783872
Strain files:  511
Log data files:  511



In [ ]:
# ------> Convert log data lists to numpy arrays of floats

for seg in range(len(strain_segments)):
    for i in range(len(logdata_segments[seg]["class " + sample_class])):
        # List of strings to list of floats
        logdata_segments[seg]["class " + sample_class][i] = [float(x) for x in logdata_segments[seg]["class " + sample_class][i]]
        # List of floats to array of floats
        logdata_segments[seg]["class " + sample_class][i] = np.array(logdata_segments[seg]["class " + sample_class][i])

    logdata_segments[seg]["class " + sample_class] = np.array(logdata_segments[seg]["class " + sample_class])

In [ ]:
# ------> Sort log data arrays by first row (ordinal number)

for seg in range(len(strain_segments)):

    # Copy main array as a temporary array
    temp_arr = logdata_segments[seg]["class " + sample_class]

    # Sort temporary array
    temp_arr = temp_arr[temp_arr[:, 0].argsort()]

    # Reasigne main array
    logdata_segments[seg]["class " + sample_class] = temp_arr

    # Delete temporary array
    del(temp_arr)
    gc.collect()

In [ ]:
# ------> Concatenate arrays of logdata segments for the selected class

if len(strain_segments) == 1:
  logdata = logdata_segments[0]["class " + sample_class]
elif len(strain_segments) == 2:
  logdata = np.concatenate((logdata_segments[0]["class " + sample_class], logdata_segments[1]["class " + sample_class]), axis=0)

#Check
print("Shape of logdata array with ordinal numbers: ", logdata.shape)

# ------> Remove ordinal numbers

logdata = np.delete(logdata, 0, 1)

# Check
print("Shape of logdata array without ordinal numbers: ", logdata.shape)

# Remark for checks:
# 1st dimension -> Number of window strain samples
# 2nd dimension -> Number of log variables for each sample

Shape of logdata array with ordinal numbers:  (511, 5)
Shape of logdata array without ordinal numbers:  (511, 4)


In [ ]:
# ------> Merge lists of strain samples and convert to numpy array
if len(strain_segments) == 1:
  samples = strain_segments[0]["class " + sample_class]
elif len(strain_segments) == 2:
  samples = strain_segments[0]["class " + sample_class] + strain_segments[1]["class " + sample_class]

samples = np.array(samples)

# Check
print("Shape of samples array:", samples.shape)

# Remark:
# 1st dimension -> Number of window strain samples
# 2nd dimension -> Window strain sample length
# 3rd dimension -> Features; time [s] and strain

Shape of samples array: (511, 3971, 2)


In [ ]:
# ------> Compute sampling frequency

fs = 1 / (np.transpose(samples[0])[0][1] - np.transpose(samples[0])[0][0])
print("Sampling frequency of window strain samples:", fs)

Sampling frequency of window strain samples: 4096.0


## 5) Samples conversion

Next cell contains functions to compute wavelet transform

In [ ]:
# @title
# ---------------------------------------------------------------
# ------> Wavelet time-frequency representation
# ---------------------------------------------------------------

def WaveletTF_transform(h, fs, fstart, fstop, delta_f, width, doplots):

    """
    This function computes the time-frequency representation
    based on a Wavelet transform of a signal (time series)

    INPUT:
            h        -> Signal
            fs       -> Sampling frequency
            fstart   -> Filter's initial frequency
            fstop    -> Filter's final frequency
            delta_f  -> Frequency bin width
            width    -> Width of the wavelet (in cycles)
            doplots  -> Do plots for checks (0: no, 1: yes)

    OUTPUT:
            timeVec  -> Vector of times
            freqVec  -> Vector of frequencies
            WL       -> Wavelet coefficients

    """

    # ------> Time vector and time sampling

    Nsamples = len(h)
    timeVec = np.arange(0, Nsamples)/fs
    ts      = 1/fs

    # ------> Frequency vector

    Nfreq = round((fstop - fstart) / delta_f) + 1
    freqVec = np.linspace(fstart, fstop, Nfreq).reshape((-1, 1))

    # ------> Initialize Wavelet Transform Matrix

    WL = np.zeros((Nfreq, Nsamples))

    #print("Wavelet Transform Matrix shape:", WL.shape)

    # ------> Compute the time-frequency representation

    signal = h
    #signal = detrend(h, axis=-1, type='linear')

    for ifre in range(Nfreq):

        # Compute the Morlet wavelet
        Morlet = Morlet_wavelet(freqVec[ifre], ts, width, doplots);

        # Apply the Morlet wavelet transform
        WLcomplex = np.convolve(signal, Morlet, mode='full')

        # Get indexes
        li = int(np.ceil(len(Morlet) / 2))
        ls = li + Nsamples
        #ls = len(WLcomplex) - int(np.flor(len(Morlet) / 2))

        # Complex coefficients
        WLcomplex = WLcomplex[li:ls]

        if doplots:

            print("Frecuency =", freqVec[ifre])
            print("++++++++++++++++++++++++++++++++++++++++")
            print("")

            plt.figure()
            plt.subplot(3, 1, 1)
            plt.plot(np.real(WLcomplex), 'r')
            plt.plot(np.imag(WLcomplex), 'b')
            plt.legend(['real', 'imag'])
            plt.box(True)
            plt.title('Frequency: ' + str(freqVec[ifre]) + ' Hz')

            plt.subplot(3, 1, 2)
            plt.plot(np.abs(WLcomplex))
            plt.legend(['magnitude'])
            plt.box(True)

            plt.subplot(3, 1, 3)
            plt.plot(np.angle(WLcomplex))
            plt.legend(['phase'])
            plt.box(True)

            plt.tight_layout()
            plt.pause(0.1)  # Pause for a short time to show the plot
            #plt.savefig('wavelet_decomposition.png')

        # Compute the magnitude
        WLmag = 2*(np.abs(WLcomplex)**2)/fs

        # Save wavelet decomposition magnitude
        WL[ifre, :] = WLmag

        WL = np.squeeze(WL)

        if doplots:

            plt.figure()
            X, Y = np.meshgrid(timeVec, freqVec)

            plt.pcolormesh(X, Y, WL[:, :], shading='gouraud')

            plt.axis([np.min(timeVec), np.max(timeVec), np.min(freqVec), np.max(freqVec)])
            plt.xlabel('Time (s)')
            plt.ylabel('Frequency (Hz)')
            plt.title('Time-Frequency representation')
            plt.box(True)
            plt.colorbar()

            plt.show()

    return timeVec, freqVec, WL

# ---------------------------------------------------------------
# ------> Morlet Wavelet time-frequency representation
# ---------------------------------------------------------------

def Morlet_wavelet(fi, ts, morlet_w, doplots):

    """
    Function to compute the Complex Morlet Wavelet for frequency "fi"
    and time "t". This will be normalized so the total energy = 1.

    INPUT:
            fi        -> Frequency
            ts        -> Sampling time
            morlet_w  -> Wavelet's width (width>= 5 is suggested)
            doplots   -> Do plots for checks (0: no, 1: yes)

    OUTPUT:
            Morlet    -> Morlet wavelet
    """

    # Frequency standard deviation
    sf = fi / morlet_w

    # Time standard deviation
    st = 1/(2*np.pi*sf)

    # Wavelet's amplitude
    A  = 1/np.sqrt(st*np.sqrt(np.pi))

    # Time array
    t = np.arange(-3.5 * st, 3.5 * st + ts, ts)

    # Compute Morlet wavelet
    Morlet =  A*np.exp(-t**2/(2*st**2))*np.exp(1j*2*np.pi*fi*t)

    if doplots:

        plt.figure()

        plt.subplot(3, 1, 1)
        plt.plot(t, np.real(Morlet), 'r', linewidth=2)
        plt.plot(t, np.imag(Morlet), 'b', linewidth=2)
        plt.ylabel('Morlet')
        plt.legend(['Real', 'Imag'])
        plt.box(True)
        plt.title('Frequency = ' + str(fi) + 'Hz')

        plt.subplot(3, 1, 2)
        plt.plot(t, np.abs(Morlet), '.-r')
        # plt.axis([-4, 4, 0, 6])
        plt.xlabel('Time (s)')
        plt.ylabel('Magnitude')

        plt.subplot(3, 1, 3)
        plt.plot(t, np.angle(Morlet), '.-b')
        # plt.axis([-4, 4, -4, 4])
        plt.xlabel('Time (s)')
        plt.ylabel('Angle')

        plt.tight_layout()
        plt.show()

    return Morlet

In [ ]:
# ------> Select samples CCSNe waveform model to convert

os.chdir(data_dir)

for folder in glob.glob(detector + "*"):

    if "_TF_samples" in folder:
      continue
    else:
      seg_subdir = data_dir + folder + "/"
      #print("GPS segment:", folder[3:])
      #print("---------------------------")
      #print(seg_subdir)

    os.chdir(seg_subdir)

    for subfolder in glob.glob("wfclass_"+sample_class):

      seg_subsubdir = seg_subdir + subfolder
      #print(seg_subsubdir)

      os.chdir(seg_subsubdir)

      Models = []

      for subsubfolder in glob.glob("*_*"):
        Models.append(subsubfolder)

      print("Available CCSNe waveform models of class", sample_class)

      for i in range(len(Models)):
        print(Models[i], "  |  Input option :", i)
        print("")

    break

nfile_i = input("======> Enter your option: ")
nfile_i = int(nfile_i)
print("")

# ------> Create folders to save samples

dir_TFsamples = data_dir

# Folder for detector
dir_TFsamples = dir_TFsamples + detector + "_TF_samples_c" + sample_class

if not os.path.isdir(dir_TFsamples):
    os.makedirs(dir_TFsamples)

# Subfolder for model

dir_TFsamples = dir_TFsamples + "/" + Models[nfile_i]

if not os.path.isdir(dir_TFsamples):
    os.makedirs(dir_TFsamples)

# ------> Set location of the folder

os.chdir(dir_TFsamples)
print("Directory to save images:", dir_TFsamples)

Available CCSNe waveform models of class 3
Cerda-Duran_2013   |  Input option : 0

======> Enter your option: 0

Directory to save images: /content/drive/MyDrive/Colab Notebooks/GitHub/CCSNeHFGW_ResNetClass/Datasets/NumWf/Distance_10Kpc/H1_TF_samples_c3/Cerda-Duran_2013


In [ ]:
# ------> Save new log data file

np.save('log_TFsamples', logdata)

In [ ]:
# ------> Convert samples to images and save them

#for i in range(len(samples)):
for i in range(100): # For debugging/testing

    # Transpose sample array
    sample = np.transpose(samples[i])

    # Print the current sample's number being converted
    print("Converting window sample ", i)

    # Generate TF representation
    time, freq, Sxx = WaveletTF_transform(sample[1], fs, 10, 2000, 10, 7, 0)

    # Plot and save TF representation
    plt.figure(i, figsize=(1,1))
    plt.pcolormesh(time+sample[0][0], freq, Sxx, shading='gouraud')
    plt.xlabel('Time [s]'); plt.ylabel('Freq [Hz]')
    plt.axis("off")

    TF_sample = io.BytesIO()
    plt.savefig(TF_sample, bbox_inches='tight', pad_inches=0, format='jpg')

    # Convert matplotlib figure to PIL image
    img = Image.open(TF_sample)

    # Resize PIL image
    img_resized = img.resize((64, 64))

    TF_sample.close()
    #!rm TF_sample

    # Convert PIL resized image to RGB
    if img_resized.mode != 'RGB':
        img_resized = img_resized.convert('RGB')

    sample_number = str(i).zfill(6)

    # Save PIL resized RGB image
    img_resized.save('TF_sample_' + sample_number + '.jpg')

    # Save PIL resized RGB image as numpy array
    img_array = np.array(img_resized)
    np.save('TF_sample_' + sample_number, img_array)

    #plt.show()

    plt.figure(i).clear()
    plt.close()
    gc.collect()

Converting window sample  0


<ipython-input-31-70c9f80e4546>:150: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t = np.arange(-3.5 * st, 3.5 * st + ts, ts)


Converting window sample  1


<ipython-input-31-70c9f80e4546>:150: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t = np.arange(-3.5 * st, 3.5 * st + ts, ts)


Converting window sample  2


...

<ipython-input-31-70c9f80e4546>:150: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t = np.arange(-3.5 * st, 3.5 * st + ts, ts)


Converting window sample  98


<ipython-input-31-70c9f80e4546>:150: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t = np.arange(-3.5 * st, 3.5 * st + ts, ts)


Converting window sample  99


<ipython-input-31-70c9f80e4546>:150: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t = np.arange(-3.5 * st, 3.5 * st + ts, ts)
